In [54]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import regex as re

from sklearn.model_selection import train_test_split

# TODO

- Solve the potential data leakage on the dataset=

# 1. Solving Potential Data Leakage on the Dataset

- I thought on the problem that state and parking_options features having high feature importances.
- I noticed that I might have caused data leakage. Due to the fact that I fill the null values and encode the categorical features according to the target feature, this might be causing data leakage.
- In order to solve this I will first split the train and test sets and find the mappings for null filling the null values and encoding categorical features from the train set. Then I will apply the same mappings for the test set, just like in scaling the dataset.
- Therefore, I need to process the data again. However, I will skip the explanations this time until null value management and encoding categorical features

## 1.1 Preprocess Data

### Utils for section 1.1.1

In [55]:
def find_iqr_and_fences(df, feature):
    iqr = df[feature].quantile(0.75) - df[feature].quantile(0.25)
    upper_fence = df[feature].quantile(0.75) + 1.5 * iqr
    lower_fence = df[feature].quantile(0.25) - 1.5 * iqr
    
    print(f"{feature} feature's IQR: {iqr}")    
    print(f"Samples above with value {upper_fence} should be dropped")
    print(f"Amount of outliers detected in upper region: {df[df[feature] > upper_fence].shape[0]}\n")
    
    print(f"Samples below with value {lower_fence} should be dropped")
    print(f"Amount of outliers detected in lower region: {df[df[feature] < lower_fence].shape[0]}")
    
    return lower_fence, upper_fence

### 1.1.1 Parts that are not related with the leakage

In [56]:
df = pd.read_csv("main_raw.csv")

# added parking_options feature here
df = df.drop(columns=["id", "url", "region_url", "image_url", "parking_options", "region"])
df = df.drop_duplicates(keep="first")

In [57]:
# Outlier Management

df = df[df.price >= 400]
lower, upper = find_iqr_and_fences(df, "price")
df = df[df.price <= upper]

lower, upper = find_iqr_and_fences(df, "sqfeet")
df = df[(lower <= df.sqfeet) & (df.sqfeet <= upper)]

lower, upper = find_iqr_and_fences(df, "beds")
df = df[(lower <= df.beds) & (df.beds <= 5)]

df.baths = df.baths.apply(np.ceil).astype(int)

price feature's IQR: 645.0
Samples above with value 2467.5 should be dropped
Amount of outliers detected in upper region: 11038

Samples below with value -112.5 should be dropped
Amount of outliers detected in lower region: 0
sqfeet feature's IQR: 398.0
Samples above with value 1747.0 should be dropped
Amount of outliers detected in upper region: 7100

Samples below with value 155.0 should be dropped
Amount of outliers detected in lower region: 473
beds feature's IQR: 1.0
Samples above with value 3.5 should be dropped
Amount of outliers detected in upper region: 3706

Samples below with value -0.5 should be dropped
Amount of outliers detected in lower region: 0


In [58]:
# some other parts

df = df.dropna(subset="description")
df["lat"].fillna(df["lat"].mean(), inplace=True)
df["long"].fillna(df["long"].mean(), inplace=True)
df = df.dropna(subset="state")

df["pets_allowed"] = df.cats_allowed | df.dogs_allowed
df.drop(columns=["cats_allowed","dogs_allowed"], inplace=True)

In [59]:
# description column feature engineering

count_dict = {}
for row in tqdm(df.description):
    words = row.lower().strip().split()
    words = [re.sub("[^\w\s]", "", w) for w in words]

    for word in words:
        if word in count_dict:
            count_dict[word] += 1
        else:
            count_dict[word] = 1

count_dict = dict(sorted(count_dict.items(), key=lambda x:x[1], reverse=True))
count_dict

df["has_pool"] = df.description.str.lower().str.contains("pool|swim").astype(int)
df["has_sports"] = df.description.str.contains("fitness|gym|basketball|spa|tennis").astype(int)
df["has_park"] = df.description.str.lower().str.replace("parking", "").str.contains("park|garden").astype(int)
df["has_shopping"] = df.description.str.contains("mall|shop|market|grocery|store|downtown").astype(int)
df["has_transportation"] = df.description.str.contains("transportation|airport|station|bus|train").astype(int)

df.drop(columns=["description"],inplace=True)

100%|████████████████████████████████████████████████████████████████████████| 195680/195680 [01:30<00:00, 2161.32it/s]


### 1.1.2 Parts that are related with the leakage (nulls and encoding)

In [60]:
# I intentionally passed the X as df, because I need the target feature in the following parts
train_set, test_set, _, _ = train_test_split(df, df["price"], test_size=0.2, random_state=42)

### 1.1.2.1 null values

- there are two features need to be handled here, laundry_options and parking_options. However, I decided to drop parking_options because its 40% null
- dropping parking_options was done at the beginning of the dropping columns part

In [61]:
def fill_nulls_train(feature, train_set):
    null_indexes = train_set[train_set[feature].isna()][[feature]].index
    lookup = train_set[["price", feature]].groupby(feature).mean().price
    
    print(lookup)

    for idx in tqdm(null_indexes):
        closest_label_idx = (lookup - train_set.loc[idx, "price"]).apply(abs).argmin()
        train_set.loc[idx, feature] = lookup.index[closest_label_idx]
        
    return lookup
        
def fill_nulls_test(feature, test_set, lookup):
    null_indexes = test_set[test_set[feature].isna()][[feature]].index
    
    print(lookup)

    for idx in tqdm(null_indexes):
        closest_label_idx = (lookup - test_set.loc[idx, "price"]).apply(abs).argmin()
        test_set.loc[idx, feature] = lookup.index[closest_label_idx]

In [62]:
train_lookup = fill_nulls_train("laundry_options", train_set)
fill_nulls_test("laundry_options", test_set, train_lookup)

laundry_options
laundry in bldg       1028.799666
laundry on site       1062.515598
no laundry on site     970.261558
w/d hookups           1037.893169
w/d in unit           1329.118266
Name: price, dtype: float64


100%|██████████████████████████████████████████████████████████████████████████| 35932/35932 [00:17<00:00, 2026.30it/s]


laundry_options
laundry in bldg       1028.799666
laundry on site       1062.515598
no laundry on site     970.261558
w/d hookups           1037.893169
w/d in unit           1329.118266
Name: price, dtype: float64


100%|████████████████████████████████████████████████████████████████████████████| 8917/8917 [00:03<00:00, 2870.14it/s]


- notice that the test set is filled with the lookup table of train set

### 1.1.2.2 encoding categorical features

In [63]:
def ordinal_encode_train(feature, train_set):
    print(f"\nBEFORE:\n{train_set[feature].value_counts()}")
    
    # finding and ordering the categories of the feature according to the average price
    # finally gathering the indexes for creating a mapper
    ordered_labels = train_set[[feature,"price"]].groupby(feature).mean().sort_values("price").index
    
    # creating a mapper to use it to map the values
    mapper = {}
    for idx, label in enumerate(ordered_labels):
        mapper[label] = idx
    
    # mapping
    train_set[feature] = train_set[feature].map(mapper)
    
    print(f"\nAFTER:\n{train_set[feature].value_counts()}")
    
    return mapper

def ordinal_encode_test(feature, test_set, mapper):
    print(f"\nBEFORE:\n{test_set[feature].value_counts()}")
    
    # mapping according to the mapper of the train set
    test_set[feature] = test_set[feature].map(mapper)
    
    print(f"\nAFTER:\n{test_set[feature].value_counts()}")

In [64]:
cat_cols = ["type", "laundry_options", "state"]

for f in cat_cols:
    mapper = ordinal_encode_train(f, train_set)
    ordinal_encode_test(f, test_set, mapper)


BEFORE:
type
apartment        133144
house             10437
townhouse          5438
condo              2763
duplex             2048
manufactured       1696
cottage/cabin       395
loft                315
flat                203
in-law              104
land                  1
Name: count, dtype: int64

AFTER:
type
5     133144
2      10437
6       5438
9       2763
3       2048
1       1696
4        395
7        315
10       203
8        104
0          1
Name: count, dtype: int64

BEFORE:
type
apartment        33337
house             2588
townhouse         1341
condo              672
duplex             521
manufactured       421
cottage/cabin       91
loft                76
flat                61
in-law              28
Name: count, dtype: int64

AFTER:
type
5     33337
2      2588
6      1341
9       672
3       521
1       421
4        91
7        76
10       61
8        28
Name: count, dtype: int64

BEFORE:
laundry_options
w/d in unit           67516
w/d hookups           28742
laun

In [65]:
train_set.isna().sum()

price                      0
type                       0
sqfeet                     0
beds                       0
baths                      0
smoking_allowed            0
wheelchair_access          0
electric_vehicle_charge    0
comes_furnished            0
laundry_options            0
lat                        0
long                       0
state                      0
pets_allowed               0
has_pool                   0
has_sports                 0
has_park                   0
has_shopping               0
has_transportation         0
dtype: int64

In [66]:
test_set.isna().sum()

price                      0
type                       0
sqfeet                     0
beds                       0
baths                      0
smoking_allowed            0
wheelchair_access          0
electric_vehicle_charge    0
comes_furnished            0
laundry_options            0
lat                        0
long                       0
state                      0
pets_allowed               0
has_pool                   0
has_sports                 0
has_park                   0
has_shopping               0
has_transportation         0
dtype: int64

In [68]:
train_set = train_set.reset_index().drop(columns=["index"])
test_set = test_set.reset_index().drop(columns=["index"])

In [69]:
train_set.to_csv("main_ordinal_updated_train.csv", index=False)
test_set.to_csv("main_ordinal_updated_test.csv", index=False)

---